In [1]:
# import the modules/libraries
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient
# NLTK
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
#stemming
from nltk.stem import PorterStemmer
#lemmatization
from nltk.stem import WordNetLemmatizer 
# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import plot_confusion_matrix
from sklearn import preprocessing

# suppress all warnings (ignore unnecessary warnings msgs)
import warnings
warnings.filterwarnings("ignore")

## Load the dataset

In [2]:
df = pd.read_csv('players_22.csv')

In [3]:
df.head()


,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,...,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",93,93,78000000.0,320000.0,34,...,50+3,50+3,50+3,61+3,19+3,https://cdn.sofifa.net/players/158/023/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,https://cdn.sofifa.net/teams/1369/60.png,https://cdn.sofifa.net/flags/ar.png
1,188545,https://sofifa.com/player/188545/robert-lewand...,R. Lewandowski,Robert Lewandowski,ST,92,92,119500000.0,270000.0,32,...,60+3,60+3,60+3,61+3,19+3,https://cdn.sofifa.net/players/188/545/22_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,https://cdn.sofifa.net/teams/1353/60.png,https://cdn.sofifa.net/flags/pl.png
2,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",91,91,45000000.0,270000.0,36,...,53+3,53+3,53+3,60+3,20+3,https://cdn.sofifa.net/players/020/801/22_120.png,https://cdn.sofifa.net/teams/11/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1354/60.png,https://cdn.sofifa.net/flags/pt.png
3,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Júnior,"LW, CAM",91,91,129000000.0,270000.0,29,...,50+3,50+3,50+3,62+3,20+3,https://cdn.sofifa.net/players/190/871/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,NaN,https://cdn.sofifa.net/flags/br.png
4,192985,https://sofifa.com/player/192985/kevin-de-bruy...,K. De Bruyne,Kevin De Bruyne,"CM, CAM",91,91,125500000.0,350000.0,30,...,69+3,69+3,69+3,75+3,21+3,https://cdn.sofifa.net/players/192/985/22_120.png,https://cdn.sofifa.net/teams/10/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1325/60.png,https://cdn.sofifa.net/flags/be.png


In [4]:
df.shape

(19239, 110)

In [5]:
df=df.drop(['sofifa_id'],axis=1)

In [6]:
df=df.fillna(0)
df

,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,...,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url
0,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",93,93,78000000.0,320000.0,34,1987-06-24,...,50+3,50+3,50+3,61+3,19+3,https://cdn.sofifa.net/players/158/023/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,https://cdn.sofifa.net/teams/1369/60.png,https://cdn.sofifa.net/flags/ar.png
1,https://sofifa.com/player/188545/robert-lewand...,R. Lewandowski,Robert Lewandowski,ST,92,92,119500000.0,270000.0,32,1988-08-21,...,60+3,60+3,60+3,61+3,19+3,https://cdn.sofifa.net/players/188/545/22_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,https://cdn.sofifa.net/teams/1353/60.png,https://cdn.sofifa.net/flags/pl.png
2,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",91,91,45000000.0,270000.0,36,1985-02-05,...,53+3,53+3,53+3,60+3,20+3,https://cdn.sofifa.net/players/020/801/22_120.png,https://cdn.sofifa.net/teams/11/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1354/60.png,https://cdn.sofifa.net/flags/pt.png
3,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Júnior,"LW, CAM",91,91,129000000.0,270000.0,29,1992-02-05,...,50+3,50+3,50+3,62+3,20+3,https://cdn.sofifa.net/players/190/871/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,0,https://cdn.sofifa.net/flags/br.png
4,https://sofifa.com/player/192985/kevin-de-bruy...,K. De Bruyne,Kevin De Bruyne,"CM, CAM",91,91,125500000.0,350000.0,30,1991-06-28,...,69+3,69+3,69+3,75+3,21+3,https://cdn.sofifa.net/players/192/985/22_120.png,https://cdn.sofifa.net/teams/10/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1325/60.png,https://cdn.sofifa.net/flags/be.png
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19234,https://sofifa.com/player/261962/defu-song/220002,Song Defu,宋德福,CDM,47,52,70000.0,1000.0,22,1999-01-02,...,46+2,46+2,46+2,48+2,15+2,https://cdn.sofifa.net/players/261/962/22_120.png,https://cdn.sofifa.net/teams/112541/60.png,https://cdn.sofifa.net/flags/cn.png,0,https://cdn.sofifa.net/flags/cn.png
19235,https://sofifa.com/player/262040/caoimhin-port...,C. Porter,Caoimhin Porter,CM,47,59,110000.0,500.0,19,2002-03-18,...,44+2,44+2,44+2,48+2,14+2,https://cdn.sofifa.net/players/262/040/22_120.png,https://cdn.sofifa.net/teams/445/60.png,https://cdn.sofifa.net/flags/ie.png,0,https://cdn.sofifa.net/flags/ie.png
19236,https://sofifa.com/player/262760/nathan-logue/...,N. Logue,Nathan Logue-Cunningham,CM,47,55,100000.0,500.0,21,2000-02-07,...,45+2,45+2,45+2,47+2,12+2,https://cdn.sofifa.net/players/262/760/22_120.png,https://cdn.sofifa.net/teams/111131/60.png,https://cdn.sofifa.net/flags/ie.png,0,https://cdn.sofifa.net/flags/ie.png
19237,https://sofifa.com/player/262820/luke-rudden/2...,L. Rudden,Luke Rudden,ST,47,60,110000.0,500.0,19,2002-04-23,...,26+2,26+2,26+2,32+2,15+2,https://cdn.sofifa.net/players/262/820/22_120.png,https://cdn.sofifa.net/teams/111131/60.png,https://cdn.sofifa.net/flags/ie.png,0,https://cdn.sofifa.net/flags/ie.png


In [7]:
for i in range(len(df.columns)):
    if df[df.columns[i]].isnull()[1]==1:
        print(i,df[df.columns[i]].isnull()[1])


In [8]:
df.shape[0]

19239

In [9]:
#df = df[:2000]
df2=df[['short_name','long_name','overall','age','value_eur','wage_eur','player_face_url','player_url']]



In [10]:
import json
records = json.loads(df2.T.to_json()).values()


## Inserting data to mongodb

In [11]:

# import datetime

# # 152.70.93.112 is the local mongodb address installed
#client = MongoClient('mongodb://152.70.93.112/')


#db = client['fifa18012835']

# #text_prediction is the collection (table) name in our mongodb database
#players = db['Players']


# # Insert one record
#players.insert_many(records)

In [12]:
import datetime

# 152.70.93.112 is the local mongodb address installed
client = MongoClient('mongodb://152.70.93.112/')


db = client['fifa18012835'] 

#players is the collection (table) name in our mongodb database
players = db['Players']
#User input
plage = 20
ploverall=90
results = players.find({'age':{'$gte':plage},'overall':{'$gte':ploverall}}) # above 20 years old, above 90 overall stat

for data in results:
    long_name = data['long_name']
    player_age=data['age']
    overall = data['overall']
    player_face_url=data['player_face_url']
    value=float(data['value_eur'])
    wage=float(data['wage_eur'])
    
    print(long_name,player_age,overall,value,wage,player_face_url)

Lionel Andrés Messi Cuccittini 34 93 78000000.0 320000.0 https://cdn.sofifa.net/players/158/023/22_120.png
Robert Lewandowski 32 92 119500000.0 270000.0 https://cdn.sofifa.net/players/188/545/22_120.png
Cristiano Ronaldo dos Santos Aveiro 36 91 45000000.0 270000.0 https://cdn.sofifa.net/players/020/801/22_120.png
Neymar da Silva Santos Júnior 29 91 129000000.0 270000.0 https://cdn.sofifa.net/players/190/871/22_120.png
Kevin De Bruyne 30 91 125500000.0 350000.0 https://cdn.sofifa.net/players/192/985/22_120.png
Jan Oblak 28 91 112000000.0 130000.0 https://cdn.sofifa.net/players/200/389/22_120.png
Kylian Mbappé Lottin 22 91 194000000.0 230000.0 https://cdn.sofifa.net/players/231/747/22_120.png
Manuel Peter Neuer 35 90 13500000.0 86000.0 https://cdn.sofifa.net/players/167/495/22_120.png
Marc-André ter Stegen 29 90 99000000.0 250000.0 https://cdn.sofifa.net/players/192/448/22_120.png
Harry Kane 27 90 129500000.0 240000.0 https://cdn.sofifa.net/players/202/126/22_120.png
N'Golo Kanté 30 90 1

In [13]:
##User input
plname='Mbapp'
last_predictions_results = players.find({'short_name':{'$regex': plname}})

for data in last_predictions_results:
    long_name = data['long_name']
    player_age=data['age']
    overall = data['overall']
    player_face_url=data['player_face_url']
    value=data['value_eur']
    wage=data['wage_eur']
    print(long_name,player_age,overall,str(value/10000)[:-4]+'M',str(wage/1000)[:-3]+'M',player_face_url)

Kylian Mbappé Lottin 22 91 194M 23M https://cdn.sofifa.net/players/231/747/22_120.png
